In [5]:
import os
import keras
import random
from keras import backend as K
import tensorflow as tf
from keras.layers.core import Dense, Activation
from keras import models
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [2]:
nsfw_images = [fn for fn in os.listdir(r'D:\Data\Train\NSFW') if fn.endswith('.jpg')]
sfw_images = [fn for fn in os.listdir(r'D:\Data\Train\SFW') if fn.endswith('.jpg')]
train_dir = r'D:\Data\Train'
validation_dir = r'D:\Data\Validation'

In [3]:
mobile = keras.applications.mobilenet.MobileNet()
"""def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)"""

"def prepare_image(file):\n    img_path = ''\n    img = image.load_img(img_path + file, target_size=(224, 224))\n    img_array = image.img_to_array(img)\n    img_array_expanded_dims = np.expand_dims(img_array, axis=0)\n    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)"

In [6]:
base_model=MobileNet(weights='imagenet',include_top=False, input_shape = (224, 224, 3)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dropout(0.2)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

In [7]:
callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
callback_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='modelMOBILE.{epoch:02d}-{val_loss:.6f}.h5', save_weights_only=True, 
                                                               monitor='val_acc', mode='max', save_best_only=False)

In [8]:
"""for layer in model.layers:
    layer.trainable=False"""
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [9]:
model.compile(optimizer=Adam(lr=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                rescale = 1.0/255.) #included in our dependencies

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=30,
                                                 class_mode='categorical',
                                                 shuffle=True)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                rescale = 1.0/255.)

validation_generator =validation_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 30,
                                                          class_mode  = 'categorical', 
                                                          target_size = (224, 224))

Found 6394 images belonging to 2 classes.
Found 2137 images belonging to 2 classes.


In [12]:
batch_size = 30
#step_size_train=train_generator.n//train_generator.batch_size
#step_size_valid=validation_generator.n//validation_generator.batch_size
history = model.fit(train_generator,
          validation_data=validation_generator,
          steps_per_epoch=int(6394/batch_size),
          validation_steps=int(2137/batch_size),
          epochs=7,
         callbacks=[callback_early_stopping, callback_reduce_lr, callback_model_checkpoint])

Epoch 1/7
153/213 [====================>.........] - ETA: 2:01 - loss: 0.5798 - accuracy: 0.6895

C:\Users\Harshinee Sriram\anaconda3\lib\site-packages\PIL\Image.py:2832: DecompressionBombWarning: Image size (147659204 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


213/213 [==============================] - 573s 3s/step - loss: 0.5489 - accuracy: 0.7143 - val_loss: 1.0315 - val_accuracy: 0.4906
Epoch 2/7
213/213 [==============================] - 377s 2s/step - loss: 0.3710 - accuracy: 0.8415 - val_loss: 1.1885 - val_accuracy: 0.4995
Epoch 3/7
213/213 [==============================] - 394s 2s/step - loss: 0.2987 - accuracy: 0.8732 - val_loss: 0.3462 - val_accuracy: 0.8502
Epoch 4/7
186/213 [=========================>....] - ETA: 1:08 - loss: 0.2337 - accuracy: 0.9075

KeyboardInterrupt: 

In [10]:
model.save('MobileNetV1.h5')